<a href="https://colab.research.google.com/github/SangeerthanaBalasubramaniam/Data-Science/blob/main/Categorical_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Pre-processing

In [2]:
# Define the headers since the data does not have any
# custom header
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration", "num_doors", "body_style", "drive_wheels", 
           "engine_location", "wheel_base", "length", "width", "height", "curb_weight", "engine_type", "num_cylinders", "engine_size",
           "fuel_system", "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]

In [3]:
# Read in the CSV file and convert "?" to NaN
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data", header=None, names=headers, na_values="?" )

In [4]:
df.dtypes

symboling              int64
normalized_losses    float64
make                  object
fuel_type             object
aspiration            object
num_doors             object
body_style            object
drive_wheels          object
engine_location       object
wheel_base           float64
length               float64
width                float64
height               float64
curb_weight            int64
engine_type           object
num_cylinders         object
engine_size            int64
fuel_system           object
bore                 float64
stroke               float64
compression_ratio    float64
horsepower           float64
peak_rpm             float64
city_mpg               int64
highway_mpg            int64
price                float64
dtype: object

In [5]:
obj_df = df.select_dtypes(include=['object']).copy()
obj_df.head()

,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,engine_type,num_cylinders,fuel_system
0,alfa-romero,gas,std,two,convertible,rwd,front,dohc,four,mpfi
1,alfa-romero,gas,std,two,convertible,rwd,front,dohc,four,mpfi
2,alfa-romero,gas,std,two,hatchback,rwd,front,ohcv,six,mpfi
3,audi,gas,std,four,sedan,fwd,front,ohc,four,mpfi
4,audi,gas,std,four,sedan,4wd,front,ohc,five,mpfi


In [6]:
obj_df[obj_df.isnull().any(axis=1)]

,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,engine_type,num_cylinders,fuel_system
27,dodge,gas,turbo,NaN,sedan,fwd,front,ohc,four,mpfi
63,mazda,diesel,std,NaN,sedan,fwd,front,ohc,four,idi


In [7]:
obj_df["num_doors"].value_counts()

four    114
two      89
Name: num_doors, dtype: int64

In [8]:
# Fill with four
obj_df = obj_df.fillna({"num_doors" : "four"}) #entries having nan will have/ replace by four now

# Find and replace

In [9]:
obj_df["num_cylinders"].value_counts()

four      159
six        24
five       11
eight       5
two         4
three       1
twelve      1
Name: num_cylinders, dtype: int64

In [10]:
# entries in num_doors is in string, to make it into int. 
# replace all str into numbers.
cleanup_nums = {"num_doors": {"four": 4, "two": 2}, 
                "num_cylinders": {"four": 4, "six": 6, "five": 5, "eight": 8, "two": 2, "twelve": 12, "three":3 }
                }

In [11]:
# Replace
obj_df.replace(cleanup_nums, inplace=True) #datatype of num_doors and num_cyclinders is converted to categorical data.

In [12]:
obj_df.dtypes #datatype of num_cylinder and num_doors is int.

make               object
fuel_type          object
aspiration         object
num_doors           int64
body_style         object
drive_wheels       object
engine_location    object
engine_type        object
num_cylinders       int64
fuel_system        object
dtype: object

# Label Encoding

Convert each value in a column to a number.

Shortcut : Convert column to 'category' then do label encoding to it

In [13]:
# convert body style into category.
obj_df['body_style'] = obj_df['body_style'].astype('category') #astype->explicit typecasting.
obj_df.dtypes

make                 object
fuel_type            object
aspiration           object
num_doors             int64
body_style         category
drive_wheels         object
engine_location      object
engine_type          object
num_cylinders         int64
fuel_system          object
dtype: object

Assign encoded variable to a new column using cat.codes

In [14]:
obj_df['body_style_cat'] = obj_df['body_style'].cat.codes
obj_df.head()
# each category is given on a type.

,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,engine_type,num_cylinders,fuel_system,body_style_cat
0,alfa-romero,gas,std,2,convertible,rwd,front,dohc,4,mpfi,0
1,alfa-romero,gas,std,2,convertible,rwd,front,dohc,4,mpfi,0
2,alfa-romero,gas,std,2,hatchback,rwd,front,ohcv,6,mpfi,2
3,audi,gas,std,4,sedan,fwd,front,ohc,4,mpfi,3
4,audi,gas,std,4,sedan,4wd,front,ohc,5,mpfi,3


Disadvantage: Numerical values can be misinterpreted as having ordering respective to the value, i.e 0<1<2 etc.

# One-hot encoding

get_dummies()

In [15]:
pd.get_dummies(obj_df, columns=['drive_wheels']).head() # for each unique value in a column, a new column is created and value for new column is 0 or 1 depending on the value.
# for 'drive_wheels', we have three values fwd, 4wd and rwd. So, three columns will be created and 0 or 1 is inserted based on what type is it.

,make,fuel_type,aspiration,num_doors,body_style,engine_location,engine_type,num_cylinders,fuel_system,body_style_cat,drive_wheels_4wd,drive_wheels_fwd,drive_wheels_rwd
0,alfa-romero,gas,std,2,convertible,front,dohc,4,mpfi,0,0,0,1
1,alfa-romero,gas,std,2,convertible,front,dohc,4,mpfi,0,0,0,1
2,alfa-romero,gas,std,2,hatchback,front,ohcv,6,mpfi,2,0,0,1
3,audi,gas,std,4,sedan,front,ohc,4,mpfi,3,0,1,0
4,audi,gas,std,4,sedan,front,ohc,5,mpfi,3,1,0,0


Disadvantage: Number of columns increases proportional to the number of unique values in the column.

# Custom binary encoding

If we need to only filter for a broad category of input values in a column based on a common pattern:

In [16]:
obj_df['engine_type'].value_counts()

ohc      148
ohcf      15
ohcv      13
l         12
dohc      12
rotor      4
dohcv      1
Name: engine_type, dtype: int64

In [17]:
obj_df['OHC_Code'] = np.where(obj_df['engine_type'].str.contains('ohc'),1,0)

In [18]:
obj_df[['make', 'engine_type', 'OHC_Code']].head()

,make,engine_type,OHC_Code
0,alfa-romero,dohc,1
1,alfa-romero,dohc,1
2,alfa-romero,ohcv,1
3,audi,ohc,1
4,audi,ohc,1


# Using Scikit-learn

LabelEncoder

In [19]:
from sklearn.preprocessing import LabelEncoder

In [20]:
lb_make = LabelEncoder()
obj_df['make_code'] = lb_make.fit_transform(obj_df['make'])
obj_df[['make', 'make_code']].head()

,make,make_code
0,alfa-romero,0
1,alfa-romero,0
2,alfa-romero,0
3,audi,1
4,audi,1


LabelBinarizer

In [21]:
from sklearn.preprocessing import LabelBinarizer

In [22]:
lb_style = LabelBinarizer()
lb_results = lb_style.fit_transform(obj_df['body_style'])
pd.DataFrame(lb_results, columns=lb_style.classes_).head()

,convertible,hardtop,hatchback,sedan,wagon
0,1,0,0,0,0
1,1,0,0,0,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,0,1,0
